<a href="https://colab.research.google.com/github/muntazir02/maven/blob/main/Module_3_semantic_search_06_03_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai

2023-06-03 16:21:01.848326: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 16:21:03.536138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-03 16:21:05.675823: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-03 16:21:05.676365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [4]:
!pip install -U sentence-transformers rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a847ae05ee7927c9ee3bac301f3935a77ac30e1eb6e585ce9c2d03933db4d3f4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

Read the data for Paris Hotels

In [6]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/paris_02_11_23.csv')

In [7]:
df.shape

(28694, 18)

In [8]:
df.name.value_counts()

Hotel d'Angleterre, Saint Germain des Pres              960
InterContinental Paris - Le Grand                       800
Hotel Marceau Champs Elysees                            800
Sonder Atala Champs-Élysées                             800
Hotel Dauphine Saint Germain                            800
                                                       ... 
Hotel B55                                                94
Les Jardins du Faubourg                                  78
Ibis Styles Paris Meteor Avenue de la Porte d'Italie     62
Hotel Maxim Folies                                       54
Mama Shelter Paris West                                  40
Name: name, Length: 156, dtype: int64

In [9]:
df=df.drop_duplicates()

In [ ]:
#df.to_csv('/content/drive/MyDrive/Semantic_Search/paris_02_11_23.csv',index=False)

In [10]:
df.shape

(11990, 18)

In [11]:
df.name.value_counts()

InterContinental Paris - Le Grand                       80
Okko Hotels Paris Porte de Versailles                   80
Citadines Montmartre Paris                              80
Hotel Montfleuri                                        80
Hotel Cayre                                             80
                                                        ..
Seven Hotel Paris                                       32
Ibis Styles Paris Meteor Avenue de la Porte d'Italie    31
Hotel Du Sentier                                        28
Hotel Maxim Folies                                      27
Mama Shelter Paris West                                 20
Name: name, Length: 156, dtype: int64

In [12]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [13]:

df["combined"] = (
    "title: " + df.title.str.strip()+"; Content: " + df.text.str.strip() +"; desc: "+ df.text.str.strip()
)

In [14]:
import re

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x))) 

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))


In [15]:
embedder = SentenceTransformer('all-mpnet-base-v2')

In [16]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

GPU Found!


In [17]:
df_combined.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...


In [18]:
sample = df_combined[:10]

In [19]:
embedder =  embedder.to('cpu')

startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 3.7542612552642822


<ipython-input-19-e009c77fa025>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [20]:

embedder =  embedder.to('cuda')
startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 0.22313666343688965


<ipython-input-20-290f30ba95f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [21]:
sample.to_pickle('/content/drive/MyDrive/Colab Notebooks/df.pkl')    #to save the dataframe, df to 123.pkl

In [22]:
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/df.pkl') #to load 123.pkl back to the dataframe df

In [23]:
# df1.embedding[0]

In [24]:
query = 'close to Hotel Malte'
          
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
corpus_embeddings = embedder.encode(df_with_embedding.combined,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

In [27]:
df_with_embedding['similarity_2'] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

In [28]:
df_with_embedding

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,name,id,description,rating,rating_count,features,combined,embedding,similarity,similarity_2
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.012825324, 0.0029111414, 0.07...",0.103509,[0.32732528]
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.06402066, 0.0279823, -0.008169635, 0.055751...",0.094012,[0.2972932]
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[0.02479708, 0.044157468, -0.025919398, 0.0232...",0.054351,[0.17187147]
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[0.028466865, 0.061914634, -0.020466408, 0.075...",0.107607,[0.3402821]
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.024692703, 0.03586695, -0.001174005, 0.0902...",0.110647,[0.34989738]
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,/ShowUserReviews-g187147-d207742-r862073785-In...,en,MOBILE,BA9F96090F35C7A562E550825BAB4B32,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not quite up to intercontinental standar...,"[0.056313634, 0.05153258, -0.012567435, 0.0707...",0.074678,[0.23615363]
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,/ShowUserReviews-g187147-d207742-r861021031-In...,en,OTHER,CB65D8DF04EEB78702BB03F39BAC6717,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title luxury in the heart of the city content ...,"[-0.020568352, 0.023891881, 0.012835298, 0.099...",0.130083,[0.41135937]
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,/ShowUserReviews-g187147-d207742-r860421664-In...,en,OTHER,9D8328D86D62AFA560367B6F81565010,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title beautiful content visited for my birthda...,"[0.04333147,

In [ ]:
cosine_similarity(corpus_embeddings, query_embedding)

array([0.10350932, 0.09401236, 0.05435053, 0.10760665, 0.1106473 ,
       0.07467833, 0.13008326, 0.10028599, 0.0694253 , 0.1468393 ],
      dtype=float32)

#Entire Data

In [ ]:
# startTime = time.time()

# df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

# executionTime = (time.time() - startTime)
# print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 234.93274855613708


In [ ]:
# df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') 

##Embedding upload

In [ ]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491498, -0.012825325, 0.0029111586, 0.07..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.06402065, 0.027982308, -0.008169623, 0.0557..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[0.02479708, 0.044157468, -0.025919398, 0.0232..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[0.028466854, 0.06191462, -0.0204664, 0.075028..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.024692703, 0.03586695, -0.001174005, 0.0902..."


In [ ]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):
    query_embedding = embedder.encode(query,show_progress_bar=True)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n))
        
    return results

In [ ]:
query = 'close to Louvre and great food nearby but not too expensive'

In [ ]:
results = search_reviews(df,query,10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# hlist = []
# for r in results.index:
#     if results.name[r] not in hlist:
#         smalldf = results.loc[results.name == results.name[r]]
#         print('Hotel name: {}'.format(results.name[r]))
#         print('Price per night: {}'.format(results.price_per_night.max()))
#         print('Description: {}'.format(results.description[r]))
#         print('Similarity Score: {}'.format(smalldf.similarity.max()))
#         print('Rating Score: {}'.format(smalldf.rating.max()))
        
#         print('\n')
#         print('Relevant Reviews:')
#         for s in smalldf.index:
#             print(smalldf.text[s])
#             print('\n')
#         print('xxx---xxx')
#         print('\n')    
#         hlist.append(results.name[r])
            

In [ ]:
# df_combined.shape[1]

In [ ]:
# df = df_combined.copy()

In [ ]:
def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))
  
  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist
      



In [ ]:
search('near Eiffel')

[{'name': 'Relais 12bis B&B',
  'score': 0.6887854,
  'rating': 5.0,
  'relevant_reviews': ['We could not have been more pleased with our eight night stay at Relais 12bis.  Leo is a gracious host, doing everything possible to make us feel welcome and comfortable.  The rooms and public areas are beautifully maintained with great attention to detail; and the breakfast delicious.  We will definitely return and recommend this home away from home to our friends.',
   "It was my family of 6's first time in Paris. Travelled with my mom, aunt, sister, brother in law and their 2-year old. We booked 3 of their rooms for 4 nights. The owner Philippe and his wife, son Leon were excellent hosts.  Beautiful historic building, upgraded rooms even better than it looks in the pictures! Loved everything about it- rooms, bathrooms and beds were so comfortable and had daily housekeeping services. The breakfasts every morning were homemade- OJ, jams, eggs, toast, cold cuts, artisan cheeses and best selecti

In [ ]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))
  
  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          sm =smalldf.rating[r].mean()
          
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "description":results.description[r],
            "relevance score": smallarr.tolist(),
            "rating": sm.tolist(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

def greet(query):
    bm25 = search(query)
    return bm25

demo = gr.Interface(fn=greet, inputs="text", outputs="json")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://64e80e7c73f55bb581.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# 
